In [ ]:
# pip install peft

In [10]:
# pip install trl

In [13]:
# pip install transformers datasets peft trl accelerate bitsandbytes

In [16]:
# pip install --upgrade transformers

In [18]:
pip install transformers==4.31.0


     ---------------------------------------- 0.0/116.9 kB ? eta -:--:--
     --- ------------------------------------ 10.2/116.9 kB ? eta -:--:--
     ------ ------------------------------ 20.5/116.9 kB 162.5 kB/s eta 0:00:01
     ------------ ------------------------ 41.0/116.9 kB 281.8 kB/s eta 0:00:01
     ---------------------- -------------- 71.7/116.9 kB 393.8 kB/s eta 0:00:01
     ------------------------------------ 116.9/116.9 kB 620.2 kB/s eta 0:00:00
     ---------------------------------------- 0.0/314.9 kB ? eta -:--:--
     -------------------------------------- 314.9/314.9 kB 9.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/7

  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [49 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\tokenizers
      copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers
      creating build\lib.win-amd64-cpython-312\tokenizers\models
      copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\models
      creating build\lib.win-amd64-cpython-312\tokenizers\decoders
      copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\decoders
      creating build\lib.win-amd64-cpython-312\tokenizers\normalizers
      copying py_src\tokenizers\normalizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\normalizers
      creating build\lib.win-amd64-cpython-312\tokenizers\pre_tokenizers
      copying py_src

In [ ]:
pip install torchvision

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from trl import SFTTrainer

In [14]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

RuntimeError: Failed to import trl.trainer.sft_trainer because of the following error (look up to see its traceback):
Failed to import transformers.image_processing_utils because of the following error (look up to see its traceback):
cannot import name 'is_torchvision_v2_available' from 'transformers.utils.import_utils' (c:\Users\Dell\miniconda3\Lib\site-packages\transformers\utils\import_utils.py)

In [ ]:
def preprocess_data(example):
    return {
        "input_text": example["support"],  # Use the context from the 'support' column as input
        "target_text": f"{example['question']} <sep> {example['correct_answer']}"  # Question <sep> Answer format
    }

# Preprocess the datasets
train_data = sciq["train"].map(preprocess_data, remove_columns=sciq["train"].column_names)
validation_data = sciq["validation"].map(preprocess_data, remove_columns=sciq["validation"].column_names)

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

c:\Users\Dell\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub\models--t5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
model_name = "potsawee/t5-large-generation-race-QuestionAnswer"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    device_map="auto",  # Automatic device mapping
    load_in_8bit=True   # Use 8-bit quantization to reduce memory usage
)

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
model_name = "potsawee/t5-large-generation-race-QuestionAnswer"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

lora_config = LoraConfig(
    r=16,  
    lora_alpha=32,  
    target_modules=["q", "v"],  
    lora_dropout=0.1,
    bias="none"
)

In [ ]:
lora_config = LoraConfig(
    r=16,  # Rank of the low-rank adaptation
    lora_alpha=32,  # Scaling factor for the LoRA weights
    target_modules=["q", "v"],  # Target query and value projection layers
    lora_dropout=0.1,  # Dropout rate for LoRA layers
    bias="none",  # No bias terms in LoRA
    task_type="SEQ_2_SEQ_LM"  # Sequence-to-sequence language model
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

Map:   0%|          | 0/11679 [00:00<?, ? examples/s]

TypeError: can only concatenate list (not "str") to list

In [ ]:
def tokenize_data(examples):
    inputs = tokenizer(
        examples["input_text"], 
        max_length=512, 
        truncation=True, 
        padding="max_length"
    )
    targets = tokenizer(
        examples["target_text"], 
        max_length=128, 
        truncation=True, 
        padding="max_length"
    )
    inputs["labels"] = targets["input_ids"]
    return inputs


In [ ]:
tokenized_train = train_data.map(tokenize_data, batched=True)
tokenized_validation = validation_data.map(tokenize_data, batched=True)

In [ ]:
tokenized_train = train_data.map(tokenize_data, batched=True)
tokenized_validation = validation_data.map(tokenize_data, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir="./t5_lora_sciq",
    evaluation_strategy="epoch",
    learning_rate=3e-4,  # Slightly higher learning rate for LoRA
    per_device_train_batch_size=4,  # Reduced batch size for memory efficiency
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_strategy="epoch",
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",
    fp16=True,  # Use mixed precision training
)

# Create the Trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    tokenizer=tokenizer,
    dataset_text_field="input_text",  # The input text is the 'support' context
    max_seq_length=512
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("./t5_lora_sciq_weights")